### Red Agents

In [3]:
%cd ..

C:\Users\c21122256\PycharmProjects\CAGE-Challenge-1\cage-challenge-1\CybORG


Scenario 1b has two rules-based Red Agents. The first is our good friend the B_lineAgent. This represents an actor who has inside information, so is able to beeline straight towards the OpServer.

In [5]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents import *
from CybORG.Shared.Actions import *
from CybORG.Agents.SimpleAgents.B_line_2112 import B_lineAgent

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario_reduced_observation_2112_manual.yaml'

env = CybORG(path,'sim')

agent = B_lineAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(16):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.2.160/28
DiscoverRemoteSystems 10.0.2.160/28
DiscoverNetworkServices 10.0.2.174
DiscoverNetworkServices 10.0.2.174
ExploitRemoteService 10.0.2.174
ExploitRemoteService 10.0.2.174
PrivilegeEscalate User1
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.16.10
DiscoverNetworkServices 10.0.16.10
ExploitRemoteService 10.0.16.10
ExploitRemoteService 10.0.16.10
PrivilegeEscalate Enterprise1
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.16.0/28
DiscoverRemoteSystems 10.0.16.0/28


IndexError: list index out of range

This agent runs along a predetermined path to the Op_Server, but is smart enough able to recover its position if interrupted. We can see below after Blue Team restores some hosts, the agent works out where the error in and re-exploits its way to the Op_Server.

In [15]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
            
    print(action)
    print('Success:',obs['success'])

Impact Op_Server0
Success: FALSE
PrivilegeEscalate Op_Server0
Success: FALSE
ExploitRemoteService 10.0.180.214
Success: FALSE
ExploitRemoteService 10.0.254.108
Success: TRUE
PrivilegeEscalate Enterprise2
Success: TRUE
DiscoverNetworkServices 10.0.180.214
Success: TRUE
ExploitRemoteService 10.0.180.214
Success: TRUE
PrivilegeEscalate Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE


The other red agent is the MeanderAgent. This performs a breadth first search on all known hosts, scanning each one in turn, before attempting a mix of exploit and privilege escalate on the rest. This is an extremely slow agent in contrast to the laser-focussed B_lineAgent.

In [16]:
agent = RedMeanderAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(46):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.177.240/28
DiscoverNetworkServices 10.0.177.242
DiscoverNetworkServices 10.0.177.253
DiscoverNetworkServices 10.0.177.246
DiscoverNetworkServices 10.0.177.241
DiscoverNetworkServices 10.0.177.248
PrivilegeEscalate User0
ExploitRemoteService 10.0.177.242
ExploitRemoteService 10.0.177.253
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.212.178
ExploitRemoteService 10.0.212.178
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.212.176/28
DiscoverNetworkServices 10.0.212.190
DiscoverNetworkServices 10.0.212.186
DiscoverNetworkServices 10.0.212.184
ExploitRemoteService 10.0.212.190
PrivilegeEscalate Enterprise1
PrivilegeEscalate User0
PrivilegeEscalate User1
ExploitRemoteService 10.0.212.186
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.212.184
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.23.49
ExploitRemoteService 10.0.23.49
PrivilegeEscalate Op_Server0
DiscoverRemoteSystems 10.0.23.48/28
DiscoverNetworkServices 10.0.23.51
Disc

The Meander Agent is also able to recover from Blue's disruption.

In [17]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(24):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(env.get_last_action('Red'))

Impact Op_Server0
ExploitRemoteService 10.0.23.49
PrivilegeEscalate Enterprise0
PrivilegeEscalate Enterprise1
PrivilegeEscalate Enterprise2
PrivilegeEscalate Op_Host0
PrivilegeEscalate Op_Host1
PrivilegeEscalate Op_Host2
PrivilegeEscalate Op_Server0
PrivilegeEscalate User0
PrivilegeEscalate User1
PrivilegeEscalate User2
PrivilegeEscalate User3
PrivilegeEscalate User4
ExploitRemoteService 10.0.212.186
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.212.178
PrivilegeEscalate Enterprise1
ExploitRemoteService 10.0.212.184
PrivilegeEscalate Enterprise2
ExploitRemoteService 10.0.23.49
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0


### Blue Agents

The BlueReactRemoveAgent will wait until it sees suspicious activity, before using remove on all the hosts it has flagged. However, due to the 5% change that Red's exploit is missed, Red will always eventually get to the Op_Server.

In [18]:
env = CybORG(path,'sim',agents={'Red':B_lineAgent})

agent = BlueReactRemoveAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Monitor
Monitor
Monitor
Monitor
Remove User1
Monitor
Monitor
Remove Enterprise1
Monitor


The BlueReactRestoreAgent is the same as the React agent above, but uses the Restore action.

In [19]:
agent = BlueReactRestoreAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Restore User1
Monitor
Restore User1
Monitor
Restore User1
Monitor
Monitor
Monitor
Monitor


### Green Agent

An important part of CybORG Scenario1b is the Green agent, which represents the users on the network. The Green Agent is very simple, it only performs a scanning action on random hosts some of the time. This is only visible by Blue Agent.

In [20]:
agent = GreenAgent()

results = env.reset('Green')
obs = results.observation
action_space = results.action_space

for i in range(12):
    print(agent.get_action(obs,action_space))

GreenPortScan
GreenPortScan
Sleep
Sleep
GreenPortScan
GreenPortScan
Sleep
GreenPortScan
GreenPortScan
Sleep
Sleep
Sleep


### Keyboard Agent

The KeyboardAgent allows a human user to manually choose actions. This is useful for getting an intuition for the scenario.

In [ ]:
from CybORG.Agents.Wrappers import RedTableWrapper, BlueTableWrapper

cyborg = CybORG(path, 'sim',agents={'Red':RedMeanderAgent})
env = BlueTableWrapper(env=cyborg, output_mode='table')
env1= BlueTableWrapper(env=cyborg, output_mode='vector')

agent = KeyboardAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(3):
    print(obs)
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation

+----------------+-------------+-------------+----------+-------------+
|     Subnet     |  IP Address |   Hostname  | Activity | Compromised |
+----------------+-------------+-------------+----------+-------------+
| 10.0.37.128/28 | 10.0.37.131 |   Defender  |   None   |      No     |
| 10.0.37.128/28 | 10.0.37.140 | Enterprise0 |   None   |      No     |
| 10.0.37.128/28 | 10.0.37.130 | Enterprise1 |   None   |      No     |
| 10.0.37.128/28 | 10.0.37.141 | Enterprise2 |   None   |      No     |
| 10.0.89.208/28 | 10.0.89.219 |   Op_Host0  |   None   |      No     |
| 10.0.89.208/28 | 10.0.89.214 |   Op_Host1  |   None   |      No     |
| 10.0.89.208/28 | 10.0.89.221 |   Op_Host2  |   None   |      No     |
| 10.0.89.208/28 | 10.0.89.218 |  Op_Server0 |   None   |      No     |
| 10.0.21.48/28  |  10.0.21.56 |    User0    |   None   |      No     |
| 10.0.21.48/28  |  10.0.21.59 |    User1    |   None   |      No     |
| 10.0.21.48/28  |  10.0.21.53 |    User2    |   None   |      N